In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from requests.exceptions import HTTPError
import sys
import math

In [3]:
# url = "http://www.nepalstock.com/"
# request = requests.get(url)
# soup = BeautifulSoup(request.text, 'lxml')
# total_transactions = soup.find(text="Total Transactions")
# total_transactions = total_transactions.parent.parent.next_sibling.next_sibling.text
# total_transactions = total_transactions.replace(",", "")
# total_iteration = math.ceil(int(total_transactions)/500)
# print(total_iteration)
# print(request.text)


In [4]:

# table_list = list()
# # subgroup = "floorsheet"
# # # url = "http://iloveyou:8000"
# # r_list = list()
# # mistake_count = 0
# # for index in range(1,total_iteration+1):
# #     url = f"http://www.nepalstock.com/main/floorsheet/index/{index}/?contract-no=&stock-symbol=&buyer=&seller=&_limit=500/"
# #     while True:
# #         try:
# #             r = requests.get(url)
# #             if r.status_code == 200 :  
# #                 break
# #         except requests.exceptions.RequestException as e:  # This is the correct syntax
# #             mistake_count += 1
# #             if mistake_count > 3:
# #                 print(e)
# #                 sys.exit()
# #                 continue
        
# #     r_list.append(r)
    
    


In [4]:
# r.url
# for x in r_list:
#     print(x.url)



In [1]:

# for r in r_list:
#     soup = BeautifulSoup(r.text, "lxml")
#     table = soup.find("table", class_="table my-table").contents
#     table = list(filter(lambda a : a!='\n', table))
#     # row = [x.text for x in table[2].find_all("td")]
#     # print(row)
    
#     for row in table[2:-3]:
#         datas = row.find_all("td")
#         row = [x.text for x in datas]
#         row[5:] = [float(x) for x in row[5:]]
#         table_list.append(row)

#     # data_json = json.dumps(table_list)
#     # print(data_json)
# columns = ['S.N.', 'Contract No','Stock Symbol', 'Buyer Broker',
#            'Seller Broker', 'Quantity', 'Rate', 'Amount']
# df = pd.DataFrame(table_list, columns=columns)
# df.to_csv("floorsheet.csv")



NameError: name 'r_list' is not defined

In [21]:


df = pd.read_csv('./datas/floorsheet_2021-06-16_Wednesday.csv')
df.head(50)
df.drop_duplicates()

,Unnamed: 0,S.N.,Symbol,Buyer,Seller,Quantity,Rate,Amount
0,0,1,API,57,16,36,450.0,16200.0
1,1,1,API,38,16,37,450.0,16650.0
2,2,1,API,51,16,100,450.0,45000.0
3,3,1,API,32,16,327,450.0,147150.0
4,4,1,UPPER,53,48,50,760.0,38000.0
...,...,...,...,...,...,...,...,...
40995,40995,82,PRIN,54,46,300,1110.0,333000.0
40996,40996,82,SHIVM,44,25,10,1565.0,15650.0
40997,40997,82,SHIVM,13,25,60,1565.0,93900.0
40998,40998,82,SHIVM,13,25,100,1565.0,156500.0


In [32]:

# df.tail(10).style.hide_index()

df[(df["Amount"] > 10000000) ].sort_values("Amount", ascending =False)

# df[(df["Amount"] > 1000000)].sort_values("Amount", ascending =False)

,Unnamed: 0,S.N.,Symbol,Buyer,Seller,Quantity,Rate,Amount
24377,24377,49,NRIC,58,54,8452,1890.0,15974280.0
17966,17966,36,RHPC,3,58,18827,600.0,11296200.0
11827,11827,24,GMFIL,58,42,19399,520.0,10087480.0


In [13]:
# c1 = df["Stock Symbol"] 
df.shape



(68000, 8)

In [30]:
c2 = df["Buyer"]=="39"
c3 = df["Seller"]== "58"
# order by broker
df[df["Symbol"] == "GBLBS"].sort_values("Amount", ascending=False).head(20)
# df[c2].groupby([ "Buyer Broker", "Stock Symbol"])["Amount"].sum().head(20).sort_values(ascending=False)

,Unnamed: 0,S.N.,Symbol,Buyer,Seller,Quantity,Rate,Amount
19400,19400,39,GBLBS,26,57,200,1240.0,248000.0
4389,4389,9,GBLBS,44,46,160,1250.0,200000.0
14965,14965,30,GBLBS,45,41,150,1238.0,185700.0
18466,18466,37,GBLBS,36,41,150,1234.0,185100.0
12496,12496,25,GBLBS,49,41,125,1236.0,154500.0
2628,2628,6,GBLBS,49,6,120,1250.0,150000.0
9373,9373,19,GBLBS,20,16,112,1245.0,139440.0
21251,21251,43,GBLBS,26,35,100,1250.0,125000.0
5979,5979,12,GBLBS,26,56,100,1245.0,124500.0
2393,2393,5,GBLBS,39,38,100,1245.0,124500.0


In [3]:
c3 = df["Buyer"]== "58"
df[c3].groupby([ "Buyer", "Symbol"])["Amount"].sum().head(20).sort_values(ascending=False)

NameError: name 'df' is not defined

In [27]:
c = df["Buyer"] == "58"
df[c].groupby([ "Buyer", "Symbol"])["Amount"].sum().head(20).sort_values(ascending=False)

Buyer  Symbol
58     ALICL     6966340.0
       API       5573860.0
       AKPL      5311454.0
       CIT       4234566.0
       BOKL      1901821.0
       AHPC      1622617.0
       CCBL      1548640.0
       CBL       1547889.0
       BFC       1241360.0
       CHL        691178.0
       ADBL       474262.0
       CBBL       443750.0
       CHCL       361000.0
       AKJCL      272566.0
       CGH        237180.0
       AIL        185060.0
       BPCL       169515.0
       BARUN      115925.0
       CFCL        91150.0
       ACLBSL      47640.0
Name: Amount, dtype: float64

In [26]:
c = df["Symbol"] == "ALICL"
df[c].groupby([ "Symbol", "Seller"])["Amount"].sum().head(20).sort_values(ascending=False)

Symbol  Seller
ALICL   34        10935500.0
        19         1962844.0
        6          1665360.0
        35         1426000.0
        28         1264900.0
        16         1192914.0
        1           893300.0
        8           801750.0
        37          571000.0
        29          567848.0
        26          561660.0
        32          495727.0
        33          427975.0
        36          420119.0
        4           358500.0
        11          185890.0
        20          156560.0
        14          142600.0
        17          141620.0
        22           71240.0
Name: Amount, dtype: float64

In [29]:
c2 = df["Seller"]== "58"
df[c2].groupby([ "Seller", "Symbol"])["Amount"].sum().head(20).sort_values(ascending=False)

Series([], Name: Amount, dtype: float64)

In [31]:
#top turnover
df[df["Symbol"] == "ALICL"].groupby(["Symbol"])["Amount"].sum().sort_values(ascending = False).head(20)
# df["Amount"].sum()

Symbol
ALICL    43924825.0
Name: Amount, dtype: float64